In [1]:
import nltk, random
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer

from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression

import sys
sys.path.append(sys.path[0] + '/..')

from aprendizajeAutomatico import entrenamiento, prediccion
from comun import archivo, cadena
from preprocesamiento import tokenizacion

# Cargando datos

In [2]:
import csv

file = open("sentimiento_ternario_es.csv", "r")
data = list(csv.reader(file, delimiter=";"))
file.close()
#print(data)

Organización de los datos del corpus como una lista de tuplas, donde el primer elemento son los tokens de palabras de los documentos y el segundo es la etiqueta (pos --> positivo, neg --> negativo)

In [3]:
X_all = []
y_all = []
cont = 0
for texto, etiqueta in data:
    if cont > 2:
        y_all.append(etiqueta)
        X_all.append(tokenizacion.crear_caracteristica(texto, nrange=(1, 4)))
    cont += 1

### Separación de datos en conjuntos de entrenamiento y prueba

In [4]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size = 0.2, random_state = 100)
vectorizer = DictVectorizer(sparse = True)
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)


Selección de modelos y validación cruzada. Se prueban algunos algoritmos de clasificación comunes:
- Máquinas de vectores soporte
- Árbol de decisión
- Naive Bayes
- Regresión logística

In [5]:
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

svc = svm.SVC()
lsvc = LinearSVC(random_state=123)
rforest = RandomForestClassifier(random_state=123)
dtree = DecisionTreeClassifier(max_depth=10, random_state=123)
gnb = GaussianNB()
lg = LogisticRegression()

#clifs = [svc, lsvc, rforest, dtree, gnb, lg]
clifs = [svc, lsvc, rforest, dtree]

modelos = {}
for clf in clifs:
    #modelos[clf.__class__.__name__] = entrenamiento.entrenar_cross_score(modelo=clf, X_train=X_train_bow, y_train=y_train, esBinario=True)
    modelos[clf.__class__.__name__] = entrenamiento.entrenar_probar_accu_score(clf, X_train, X_test, y_train, y_test, True)

In [6]:
for modelo in modelos:
    print(modelo, modelos[modelo][1])

SVC {'Precisión entrenamiento': 0.8635794743429287, 'Precisión pruebas': 0.425}
LinearSVC {'Precisión entrenamiento': 1.0, 'Precisión pruebas': 0.475}
RandomForestClassifier {'Precisión entrenamiento': 1.0, 'Precisión pruebas': 0.415}
DecisionTreeClassifier {'Precisión entrenamiento': 0.6533166458072591, 'Precisión pruebas': 0.445}


## Evaluación
Para evaluar el rendimiento de cada modelo, se utilizan varias métricas comunes:
- Precision
- Recall
- F-score
- Accuracy
- Confusion Matrix

In [7]:
print ("{:<25} {:<20}".format('Modelo', 'Prección'))
print ("{:<25} {:<20}".format('----------------', '----------------'))
for clf in clifs:
    nombre_modelo = clf.__class__.__name__
    print ("{:<25} {:<20}".format(nombre_modelo, clf.score(X_test, y_test)))

Modelo                    Prección            
----------------          ----------------    
SVC                       0.425               
LinearSVC                 0.475               
RandomForestClassifier    0.415               
DecisionTreeClassifier    0.445               


In [8]:
# F1 Score del modelo que mejor predicción tiene: SVC
from sklearn.metrics import f1_score

y_pred = svc.predict(X_test)
f1_score(y_test, y_pred, 
         average=None, 
         labels = y_all)

c:\tools\ides\anaconda\envs\entorno_prueba\lib\site-packages\sklearn\metrics\_classification.py:1495: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(


array([0.09195402, 0.09195402, 0.09195402, 0.05      , 0.09195402,
       0.58608059, 0.05      , 0.09195402, 0.58608059, 0.05      ,
       0.05      , 0.58608059, 0.05      , 0.05      , 0.05      ,
       0.09195402, 0.05      , 0.09195402, 0.09195402, 0.05      ,
       0.09195402, 0.05      , 0.09195402, 0.05      , 0.05      ,
       0.05      , 0.58608059, 0.58608059, 0.09195402, 0.58608059,
       0.58608059, 0.09195402, 0.58608059, 0.05      , 0.09195402,
       0.58608059, 0.58608059, 0.58608059, 0.09195402, 0.58608059,
       0.58608059, 0.58608059, 0.09195402, 0.09195402, 0.05      ,
       0.58608059, 0.58608059, 0.05      , 0.09195402, 0.05      ,
       0.05      , 0.05      , 0.09195402, 0.05      , 0.58608059,
       0.09195402, 0.09195402, 0.58608059, 0.58608059, 0.58608059,
       0.58608059, 0.09195402, 0.58608059, 0.58608059, 0.58608059,
       0.58608059, 0.05      , 0.58608059, 0.05      , 0.58608059,
       0.05      , 0.09195402, 0.58608059, 0.58608059, 0.58608

Buscar la mejor parametrización para el modelo SVC utilizando GridSerarchCV

In [9]:
%%time

from sklearn.model_selection import GridSearchCV

parameters = {'kernel': ('linear', 'rbf'), 'C': (1,4,8,16,32)}

svc_nuevo = svm.SVC()
clf = GridSearchCV(svc_nuevo, parameters, cv=10, n_jobs=-1)
clf.fit(X_train, y_train)

c:\tools\ides\anaconda\envs\entorno_prueba\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


CPU times: total: 859 ms
Wall time: 17.4 s


GridSearchCV(cv=10, estimator=SVC(), n_jobs=-1,
             param_grid={'C': (1, 4, 8, 16, 32), 'kernel': ('linear', 'rbf')})

In [10]:
#print(sorted(clf.cv_results_.keys()))
print(clf.best_params_)
print(clf.score(X_test, y_test))

{'C': 8, 'kernel': 'rbf'}
0.435


## Predicción de la letra de canción

Verificación de los mejores parámetros identificados en el modelo SVC

In [11]:
svc = svm.SVC(C=8, kernel='rbf')
m = svc.__class__.__name__
modelos[m] = entrenamiento.entrenar_cross_score(svc, X_train, y_train, True)

print ("{:<20} {:<20} {:<20} {:<20} {:<20}".format('Predicción 1', 'Predicción 2', 'Predicción 3', 'Predicción 4', 'Predicción 5'))
print ("{:<20} {:<20} {:<20} {:<20} {:<20}".format('----------------', '----------------', '----------------', '----------------', '----------------'))
print ("{:<20} {:<20} {:<20} {:<20} {:<20}".format(modelos[m][1]['prediccion 0'], modelos[m][1]['prediccion 1'], modelos[m][1]['prediccion 2'], modelos[m][1]['prediccion 3'], modelos[m][1]['prediccion 4']))

c:\tools\ides\anaconda\envs\entorno_prueba\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


Predicción 1         Predicción 2         Predicción 3         Predicción 4         Predicción 5        
----------------     ----------------     ----------------     ----------------     ----------------    
0.44375              0.4625               0.5                  0.475                0.4528301886792453  


Obtiene letra de canción preprocesada

In [23]:
from bdd import mongoDB_cliente
from aprendizajeAutomatico import prediccion

cantante = 'Coldplay, BTS'
cancion = 'My Universe'

canciones = mongoDB_cliente.obtenerColeccion('CANCIONES', {'cantante': cantante, 'cancion': cancion})   
letra_preprocesada = canciones[0]['letra_preprocesada']
letra_original = canciones[0]['letra']
array_letra = {}
letra_cancion_pre = ''
for letra in letra_preprocesada:
        letra_cancion_pre = letra_cancion_pre + letra + " "
array_letra = [letra_cancion_pre]
array_letra_o = [letra_original]


In [24]:
array = array_letra_o

#letra_bow = tfidf_vec.transform(array)
#prediccion = svc.predict(letra_cancion_pre)
modelo = rforest
res_prediccion, modelos = prediccion.predecirTexto(modelo, letra_cancion_pre, vectorizer, modelos, cancion, cantante)
print(res_prediccion)
     

POSITIVO


In [14]:
from logica import cancion_logica
from sentimientos import sentimientoMultiple

cancion_logica.actualizarCancionSentimientoBinario(canciones[0], res_prediccion)

'documento actualizado correctamente'

## Procesamiento de todas las letras

In [27]:
canciones = mongoDB_cliente.obtenerColeccion('CANCIONES', {'estado': 'LETRA PREPROCESADO'})

cont = 0
for c in canciones:
    cont += 1
    letra_preprocesada = c['letra_preprocesada']
    letra_original = c['letra']
    array_letra = {}
    letra_cancion_pre = ''
    for letra in letra_preprocesada:
            letra_cancion_pre = letra_cancion_pre + letra + " "
    modelo = rforest
    res_prediccion, modelos = prediccion.predecirTexto(modelo, letra_cancion_pre, vectorizer, modelos, cancion, cantante)
    cancion_logica.actualizarCancionSentimientoBinario(c, res_prediccion)
print(cont)
    

98
